# Softmax with Negative Sampling

word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method
* https://arxiv.org/abs/1402.3722

REMIND

## softmax
$$P(o|c) = \dfrac{\exp(u_o^T v_c)}{\sum_{w=1}^V \exp(u_w^T v_c)}$$
* propose: maximize $P(o|c)$
* Cost function is not convex, initialization is not matter
* update: large and sparse
    * we want to update the word vectors that actually appear! 
    * --> how to solve this?

### Solution
1. need sparse matrix update operations to only update certain columns of full embedding matrices U and V
2. nedd to keep around a hash for word vectors

paper: Distributed representaions of Words and Phrases and their Compositionality (Mikolov et al. 2013)


## Negative Sampling

Main idea: train binary logistic regressions for a true pair (center word and word in its context window) versus a couple of noise pairs (the center word paired with a random word)

* Maximize Overall objective function:
$$\begin{aligned}
J(\theta) &= \dfrac{1}{T}\sum_{t=1}^{T} J_t(\theta)\\
J(\theta) &= \log \sigma(u_o^T v_c) + \sum_{i=1}^{k} \mathbb{E}_{j \tilde{} P(w)} [\log \sigma(-u_j^T v_c)]
\end{aligned}$$
    * $T$: total num of words
    * $\sigma$: sigmoid function
    * $P(w) = {U(w)^{3/4}} / {Z}$: unigram distribution U(w) raised to the 3/4 power
    
So we maximize the probability of two words co-occurring in the first $\log \sigma(u_o^T v_c)$

And, sub sample a couple of the words from the corpus (j ~ P(w)), minimize their probability of co-occurring



### 논문 설명:
* 출발점: (w, c) 세트가 정말로 corpus data로 부터 왔는가? 
* 정의: 
    * $P(D = 1|w, c)$ (w, c)가 corpus data로부터 왔을 확률
    * $P(D = 0|w, c) = 1 - P(D = 1|w, c)$ (w, c)가 corpus data로부터 오지 않았을 확률
  
따라서, 우리의 목적은 확률$P(D = 1|w, c)$를 최대화하는 parameter $\theta$를 찾는 것이기 때문에 아래와 같은 식을 세울 수 있다.
$$\begin{aligned} &\arg \underset{\theta}{\max} \underset{(w,c) \in D}{\prod} P(D=1|w,c;\theta) \\
= &\arg \underset{\theta}{\max} \log \underset{(w,c) \in D}{\prod} P(D=1|w,c;\theta) \\
= &\arg \underset{\theta}{\max} \underset{(w,c) \in D}{\sum} \log P(D=1|w,c;\theta)
\end{aligned}$$ 

확률 $P(D=1|w,c;\theta)$은 sigmoid로 아래와 같이 정의 할 수 있다.
$$P(D=1|w,c;\theta) = \dfrac{1}{1+e^{-v_c v_w}}$$

따라서 우리의 목적은 아래와 같다.
$$\arg \underset{\theta}{\max} \underset{(w,c) \in D}{\sum} \log \dfrac{1}{1+e^{-v_c v_w}}$$

그러나 우리의 목적 함수는 trivial solution이 존재한다. 

#### 참고
trivial solution: 모든 해가 0 이어야 방정식이 풀리는거
non-trivial solution: 무수히 많은 해가 있음(보통 하나가 free variable)
homogenous, trival solution 개념 설명: https://www.youtube.com/watch?v=JlJWyWJARRU




In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter, defaultdict
torch.manual_seed(1)